<a href="https://colab.research.google.com/github/YusolCho/Game_Analysis/blob/main/LoL_OpenAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd
import numpy as np

# 블로그 내용 따라해보기 

## 1. 소환사 닉네임을 통해서 소환사 고유 id 수집

In [ ]:
api_key = 'RGAPI-482c5540-928d-4608-84ff-ae991acb7eda'
sohwan = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" +'hide on bush' +'?api_key=' + api_key  # hide on bush 는 페이커선수의 닉네임
r = requests.get(sohwan)
r.json()['id'] #소환사의 고유 id -> 자꾸 오류뜸.. 


'lgExIhAfk5O0-ZF3jWPHokAjZelS5A3kzVfbAEXfN08Cgg'

In [ ]:
type(r)

requests.models.Response

## 2. 소환사 고유 id를 통해서 소환사 랭크정보 수집

In [ ]:
# 페이커 선수의 고유 id 사용해서 랭크정보 수집
tier_url = "https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/" + r.json()['id'] +'?api_key=' + api_key
r2  = requests.get(tier_url)
r2.json() 

[{'leagueId': '446781dd-2147-3d8a-be18-f9a4f27e87bd',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'GRANDMASTER',
  'rank': 'I',
  'summonerId': 'lgExIhAfk5O0-ZF3jWPHokAjZelS5A3kzVfbAEXfN08Cgg',
  'summonerName': 'Hide on bush',
  'leaguePoints': 577,
  'wins': 154,
  'losses': 139,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStreak': False}]

## 3. 원하는 리그의 소환사 데이터 수집 
* 가장 중요한 경기 기록 데이터를 수집해보도록 한다. 

In [ ]:
#그랜드 마스터 랭크의 소환사 데이터 수집 
grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(grandmaster)#그마데이터 호출
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True)#수집한 그마데이터 index정리
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
league_df = pd.concat([league_df, league_entries_df], axis=1) #열끼리 결합

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()
league_df.to_csv('그마데이터.csv',index=False,encoding = 'cp949')#중간저장

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          700 non-null    object
 1   summonerId    700 non-null    object
 2   summonerName  700 non-null    object
 3   leaguePoints  700 non-null    object
 4   wins          700 non-null    object
 5   losses        700 non-null    object
 6   veteran       700 non-null    object
 7   inactive      700 non-null    object
 8   freshBlood    700 non-null    object
 9   hotStreak     700 non-null    object
dtypes: object(10)
memory usage: 54.8+ KB


## 4. 소환사 고유 닉네임을 이용해 유저별 고유 AccountId 수집
* 이 부분은 Summoner API에서 summonername을 사용하던 summonerId를 사용하던 노상관
* (유저의 accountId만 있으면 되기 때문)
* 이렇게 불러온 소환사 닉네임을 통해서 Step1과 Step2를 숙지한 것처럼 그랜드마스터랭크의 소환사 고유 id(accountId)를 수집합니다.

In [ ]:
for i in range(len(league_df)):
    try:
        sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(sohwan)
        
        while r.status_code == 429:
            time.sleep(5)
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)
            
        account_id = r.json()['accountId']
        league_df.iloc[i, -1] = account_id
    
    except:
        pass

## 5. 매치 데이터 수집 
* 시간 오래걸림 -> 라이엇측에서 개발자 api이용량의 한계를 걸어놈 -> 지속적으로 수집이 불가능 
* 따라서 time모듈을 활용하여 수집 OR 끊어서 수집
* 경기기록 데이터(매치 데이터) 위치 :  MATCH탭의 1번째 API
* 선행프로세스가 존재합니다. 

 

1. matchid를 얻기 : get api를 사용해야
2. api 얻딕 : 직전에 수집한 accountid를 이용
* API가 필요한 토큰 중 encryptedAccountId 부분에 4단계에서 수집한 accountId가 넣기

In [ ]:
#13시즌의 데이터만을 이용할 것이며 league_df3 => 기존에 수집한 account_id가 있는 league_df
match_info_df = pd.DataFrame()
season = str(13)
for i in range(len(league_df3)):
    try:
        match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df3['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
        r = requests.get(match0)
        
        while r.status_code == 429:
            time.sleep(5)
            match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df3['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
            r = requests.get(match0)
            
        match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
    
    except:
        print(i)

# 직접 데이터 수집해보기 

In [ ]:
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
champion_data

{'freeChampionIds': [2,
  14,
  19,
  33,
  54,
  58,
  61,
  96,
  101,
  117,
  127,
  142,
  150,
  221,
  235,
  421],
 'freeChampionIdsForNewPlayers': [222, 254, 427, 82, 131, 147, 54, 17, 18, 37],
 'maxNewPlayerLevel': 10}

In [ ]:
pip install riotwatcher 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 KB 2.8 MB/s eta 0:00:00


In [ ]:
from riotwatcher import LolWatcher, ApiError

In [ ]:
lol_watcher = LolWatcher('RGAPI-21a95d0f-29f5-47fa-8219-5b78ff79742c')
my_region = 'kr'
me = lol_watcher.summoner.by_name(my_region, 'hideonbush')
print(me)

{'id': 'lgExIhAfk5O0-ZF3jWPHokAjZelS5A3kzVfbAEXfN08Cgg', 'accountId': 'GYHT6f-zfeLAtv_-L_cFybVMwD5Y-wpSA5SZuuMs-UYw', 'puuid': '-_40s7V-TI6iyZJzU0FgYVuDnLraPFgFcYMNOw4UFII9S4-3KRVJ4UhuEWcHtCm2hboHbDMNjUO7zQ', 'name': 'Hide on bush', 'profileIconId': 6, 'revisionDate': 1678909889623, 'summonerLevel': 644}


In [ ]:
print(current_champ_list_df)

             type             format version  \
Aatrox   champion  standAloneComplex  13.5.1   
Ahri     champion  standAloneComplex  13.5.1   
Akali    champion  standAloneComplex  13.5.1   
Akshan   champion  standAloneComplex  13.5.1   
Alistar  champion  standAloneComplex  13.5.1   
...           ...                ...     ...   
Zeri     champion  standAloneComplex  13.5.1   
Ziggs    champion  standAloneComplex  13.5.1   
Zilean   champion  standAloneComplex  13.5.1   
Zoe      champion  standAloneComplex  13.5.1   
Zyra     champion  standAloneComplex  13.5.1   

                                                      data  
Aatrox   {'version': '13.5.1', 'id': 'Aatrox', 'key': '...  
Ahri     {'version': '13.5.1', 'id': 'Ahri', 'key': '10...  
Akali    {'version': '13.5.1', 'id': 'Akali', 'key': '8...  
Akshan   {'version': '13.5.1', 'id': 'Akshan', 'key': '...  
Alistar  {'version': '13.5.1', 'id': 'Alistar', 'key': ...  
...                                                    ..

In [ ]:
champion_data

{'status': {'message': 'Unauthorized', 'status_code': 401}}

# 틀  

In [ ]:
#FRAME 1.
import requests
import pandas as pd

api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'  # Replace with your own API key

# Define the base URL for the API
base_url = 'https://kr.api.riotgames.com'

# Define the API endpoint for the data you want to collect
# Example: match data endpoint
match_endpoint = '/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}'

# Define a function to make API requests
def request_data(url):
    headers = {
        'X-Riot-Token': api_key,
    }
    response = requests.get(url, headers=headers)
    return response.json()


In [ ]:
request_data(base_url+match_endpoint)

{'status': {'message': 'Bad request - Invalid routing value kr',
  'status_code': 400}}

In [ ]:
#FRAME 2.
# Define a function to process the collected data and convert to a pandas dataframe
def process_data(data):
    # Add code here to extract and process the data you need
    # Example: extract match IDs from match data
    match_ids = data['ids']
    
    # Add code here to collect additional data using the extracted IDs
    # Example: collect match details for each match ID
    match_data = []
    for match_id in match_ids:
        match_details = request_data(base_url + '/lol/match/v5/matches/' + match_id)
        match_data.append(match_details)
    
    # Add code here to process the collected data and convert to a pandas dataframe
    # Example: create dataframe of match details
    matches = []
    for match in match_data:
        match_id = match['metadata']['matchId']
        game_mode = match['info']['gameMode']
        game_duration = match['info']['gameDuration']
        # Add additional fields as needed
        matches.append({'match_id': match_id, 'game_mode': game_mode, 'game_duration': game_duration})
    match_df = pd.DataFrame(matches)
    return match_df


In [ ]:
#FRAME 3.
# Example usage: collect match data for a specific player
puuid = 'YOUR_PUUID_HERE'  # Replace with the player's PUUID
start = 0  # Replace with the starting index for the data you want to collect
count = 100  # Replace with the number of data points you want to collect

# Make API request to collect match data
match_url = base_url + match_endpoint.format(puuid=puuid, start=start, count=count)
match_data = request_data(match_url)

# Process the collected data and convert to a pandas dataframe
match_df = process_data(match_data)

# Print the resulting dataframe
print(match_df.head())

# LEAGUE-EXP-V4(티어별 소환사 정보 반환)

In [1]:
import pandas as pd

In [2]:
import requests
import pprint

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"
}

def league_v4_queue_tier_division(queue, tier, division, page_number):
    if division == 1 or tier in ["CHALLENGER", "GRANDMASTER", "GOLD"]:
        division = 'I'
    elif division == 2 :
        division = 'II'
    elif division == 3 :
        division = 'III'
    elif division == 4:
        division = 'IV'
    if queue == "solo" :
        queue = "RANKED_SOLO_5x5"
    elif queue == "free" :
        queue = "RANKED_FLEX_SR"
    url = f"https://kr.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}?page={page_number}"
    return requests.get(url, headers=request_header).json()


In [3]:
(pd.DataFrame(league_v4_queue_tier_division('free','PLATINUM', 'I',3))).head(10)

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries
0,f260434b-72a3-45d0-846c-8464b278b01b,RANKED_FLEX_SR,PLATINUM,I,H4myafiMgYRqKFy10pecahjZWJSQQ-d7GsiEbxHBAsPRndE,yamuzer,0,11,8,False,False,False,False,NaN
1,f072db37-cc38-45fb-a225-cb99f69a79d0,RANKED_FLEX_SR,PLATINUM,I,MHomhdDguGjx7ARABgJ961dCeZM6cjyEmZ65JWaViMHtaeE,SunDongLyul,100,37,22,False,False,False,False,"{'target': 2, 'wins': 0, 'losses': 1, 'progres..."
2,c17ad294-7daa-46a0-844b-553e18e5b7dd,RANKED_FLEX_SR,PLATINUM,I,xPsuD1taLugX-42O3BvbTQNmhj6P_81XT-F-1398RbDpdxE,내가바로조선걸레,12,27,21,False,False,False,True,NaN
3,bdcbfd41-dcaa-488d-8867-39b373718fba,RANKED_FLEX_SR,PLATINUM,I,vR8Wes9mnbYP_EC8JxB2eMPRVnjEixegV2XeIY0sImFCif0,착한짜장죽은짜장,10,93,48,False,False,False,False,NaN
4,64f9f9b7-012d-46d9-ab6e-2e530ed9b1e8,RANKED_FLEX_SR,PLATINUM,I,hXeN9W2imC9QIr69TCNOqTM0hclL2OaW1dXcYiLY8P86Uco,탑갱절 대가,44,42,39,False,False,False,False,NaN
5,bd45e7c1-9aab-4fa0-9dd6-5813f3bcc028,RANKED_FLEX_SR,PLATINUM,I,quUimzQIkh2O43fOdFzuy0AgirNLSfCOa3FzfyzQV7zTZGw,한국원딜러,18,27,21,False,False,False,False,NaN
6,3afb9d6a-708d-4f25-aaa2-91b946d6fbbb,RANKED_FLEX_SR,PLATINUM,I,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0,성 놈,100,77,57,False,False,False,False,"{'target': 2, 'wins': 1, 'losses': 1, 'progres..."
7,1c61ffe5-5913-4e95-9853-4305263e634f,RANKED_FLEX_SR,PLATINUM,I,sZ2tCS_5-OfkTZ5BOfHgGF1B50gg-Fv_Fazq3UrHeKOpSI4,장다혜내꺼,64,8,9,False,False,False,False,NaN
8,f8880a51-3a9e-452b-b719-19c6014d7279,RANKED_FLEX_SR,PLATINUM,I,fTMI5HQyn3_DKSrKP4KiiJv2Gh6tPwhi0uJfsHUAB-ij54A,북극여우 귀여웡,8,10,9,False,False,False,False,NaN
9,716078f7-cdc7-4fd0-a1be-8f187b53dc3b,RANKED_FLEX_SR,PLATINUM,I,-QpK9r0N5nLmS4a4IsJ-2qYY4XQSiGJ5RTyL6PZyVfDVDWc,막뚱이,19,45,41,False,False,False,False,NaN


In [4]:
summonerName_1 = '성 놈'
summonerName_2 = 'SunDongLyul'
summonerName_3 = '탑갱절 대가'
summonerName_4 = '착한짜장죽은짜장'
summonerName_5 = '한국원딜러'


# Summoner-v4(유저 정보 반환) 

In [5]:
import requests
import pprint

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"
}

def summoner_v4_by_summoner_name(summonerName):
    encodingSummonerName = summonerName
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{encodingSummonerName}"
    return requests.get(url, headers=request_header).json()

print(summoner_v4_by_summoner_name(summonerName_1)) # 결과의 'id'부분이 encryptedSummonerId에 해당됨
print(summoner_v4_by_summoner_name(summonerName_2))
print(summoner_v4_by_summoner_name(summonerName_3))
print(summoner_v4_by_summoner_name(summonerName_4))
print(summoner_v4_by_summoner_name(summonerName_5))

{'id': '61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0', 'accountId': '5Dztuv3RseKeRwrZDhGhzq8MCzJZSLdwrP-SNE65vNhG', 'puuid': 'RlJhFX2qFd31ZqywPxaw0zeTMA0Srx5PnYiubV6O0qMuCdrEzOfGSOHzwiyJwzIulFREeoV55E3wVg', 'name': '성 놈', 'profileIconId': 4690, 'revisionDate': 1678939911997, 'summonerLevel': 367}
{'id': 'MHomhdDguGjx7ARABgJ961dCeZM6cjyEmZ65JWaViMHtaeE', 'accountId': '1wPHXr3Mn5ttMBo71-0ZOTJVmqniSOcDvohFmqf8dP7Z', 'puuid': 'iJZGCFsmEfz6bllQz2fOK7VstYpJDcUjyHi38Igb7DVUkD1EFejPYU8tUYlnBA_4vIuIqwSZvwNu3g', 'name': 'SunDongLyul', 'profileIconId': 582, 'revisionDate': 1677950144000, 'summonerLevel': 222}
{'id': 'hXeN9W2imC9QIr69TCNOqTM0hclL2OaW1dXcYiLY8P86Uco', 'accountId': 'MfSAcRpLKvsclhdBdK39o4Vtu7rchCmS6asresXfFR9s', 'puuid': 'aVxn31341sAq_IWhDhLTdq3uAuM1WSw9Hky_69HiPLum3XYM0_2LIXbdKmO4rFnD7MGlz_IHQXaKrw', 'name': '탑갱절 대가', 'profileIconId': 588, 'revisionDate': 1678709414000, 'summonerLevel': 330}
{'id': 'vR8Wes9mnbYP_EC8JxB2eMPRVnjEixegV2XeIY0sImFCif0', 'accountId': 'ZNMWxl5v1JMLLJfN

In [6]:
puuid_1 = 'RlJhFX2qFd31ZqywPxaw0zeTMA0Srx5PnYiubV6O0qMuCdrEzOfGSOHzwiyJwzIulFREeoV55E3wVg'
puuid_2 = 'iJZGCFsmEfz6bllQz2fOK7VstYpJDcUjyHi38Igb7DVUkD1EFejPYU8tUYlnBA_4vIuIqwSZvwNu3g'
puuid_3 = 'aVxn31341sAq_IWhDhLTdq3uAuM1WSw9Hky_69HiPLum3XYM0_2LIXbdKmO4rFnD7MGlz_IHQXaKrw'
puuid_4 = 'b5_X9o8-jj8Vb_LsBFg-SX8w7SIDOKjBXgzbk_6Uaby2AW5iiBSoIsZGM90vV7vd_zFX_ViYEvMFKw'
puuid_5 = '9K_AQzxH4DUTWqk9350k6GYNOkr2r-E-CKDy-Hz3wzVj4fALx8wSMO18Fg5iqkBhdI0eCpyltZ3kMQ'
summonerId_1 = '61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0'
summonerId_2 = 'MHomhdDguGjx7ARABgJ961dCeZM6cjyEmZ65JWaViMHtaeE'
summonerId_3 = 'hXeN9W2imC9QIr69TCNOqTM0hclL2OaW1dXcYiLY8P86Uco'
summonerId_4 = 'vR8Wes9mnbYP_EC8JxB2eMPRVnjEixegV2XeIY0sImFCif0'
summonerId_5 = 'quUimzQIkh2O43fOdFzuy0AgirNLSfCOa3FzfyzQV7zTZGw'

# CHAMPION-MASTERY-V4(해당 유저의 챔피언 숙련도) 

In [7]:
import requests
import pprint

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}

def champion_mastery_v4(summonerid):
    encryptedSummonerId = summonerid
    url = f"https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{encryptedSummonerId}"
    return requests.get(url, headers=request_header).json()

pd.DataFrame(champion_mastery_v4(summonerId_1))
# summonerId_1,2,3,4,5 다 적용 가능 

,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,67,7,460332,1677334164000,438732,0,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
1,81,7,209871,1677163250000,188271,0,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
2,236,7,205440,1676132686000,183840,0,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
3,64,7,164007,1678542104000,142407,0,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
4,104,7,162151,1677331410000,140551,0,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
...,...,...,...,...,...,...,...,...,...
139,74,1,171,1534426672000,171,1629,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
140,201,1,165,1539610300000,165,1635,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
141,526,1,155,1612343240000,155,1645,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0
142,6,1,120,1546593892000,120,1680,False,0,61M5QeGm0_HmexsKDZlQc8uCAOcMFlT6JJ_A_gXGBF2_mF0


# 소환사의 게임id 리스트 

In [8]:
import requests
import pprint

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}

def match_v5_gamelist(summoner_puuid,start, count):
    puuid = summoner_puuid
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    return requests.get(url, headers=request_header).json()

gamelist_1 = match_v5_gamelist(puuid_1,0,100)
gamelist_1 #한번에 최대 100개 출력가능 




['KR_6407238339',
 'KR_6407219214',
 'KR_6401412528',
 'KR_6401305429',
 'KR_6401213810',
 'KR_6401117768',
 'KR_6401034366',
 'KR_6400988231',
 'KR_6399883575',
 'KR_6397100533',
 'KR_6397024569',
 'KR_6393556122',
 'KR_6393478139',
 'KR_6393354784',
 'KR_6393297515',
 'KR_6385773220',
 'KR_6385711729',
 'KR_6385643974',
 'KR_6385576355',
 'KR_6385509176',
 'KR_6385421609',
 'KR_6385318678',
 'KR_6385271903',
 'KR_6384096502',
 'KR_6384036609',
 'KR_6383968488',
 'KR_6383878637',
 'KR_6383815043',
 'KR_6383692922',
 'KR_6383616740',
 'KR_6383546445',
 'KR_6382545321',
 'KR_6382481622',
 'KR_6382474219',
 'KR_6382419305',
 'KR_6382356946',
 'KR_6380802794',
 'KR_6380716945',
 'KR_6380623926',
 'KR_6380538301',
 'KR_6380417357',
 'KR_6380312329',
 'KR_6380237675',
 'KR_6377330592',
 'KR_6377254310',
 'KR_6377199448',
 'KR_6377146391',
 'KR_6377068032',
 'KR_6376967554',
 'KR_6376884848',
 'KR_6374572822',
 'KR_6374558301',
 'KR_6374514472',
 'KR_6374464029',
 'KR_6373808302',
 'KR_63737

In [9]:
gamelist_2 = match_v5_gamelist(puuid_2,0,100)
gamelist_3 = match_v5_gamelist(puuid_3,0,100)
gamelist_4 = match_v5_gamelist(puuid_4,0,100)
gamelist_5 = match_v5_gamelist(puuid_5,0,100)


In [10]:
import time
import datetime

In [44]:
import requests
import pandas as pd

# Riot API key
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}
# 사용자의 puuid와 match id 리스트
puuid = puuid_1
match_id_list = gamelist_1

# 각 match id에 해당하는 match 정보 가져오기
match_data_list = []
for match_id in match_id_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    match_data_list.append(response.json())

# match 정보를 데이터프레임으로 변환하기
match_df_list = []
for match_data in match_data_list:
    match_df = pd.json_normalize(match_data)
    match_df_list.append(match_df)
    
# 모든 match 정보를 하나의 데이터프레임으로 합치기
match_df_1 = pd.concat(match_df_list, axis=0)


In [45]:
match_df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 0
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   metadata.dataVersion     86 non-null     object 
 1   metadata.matchId         86 non-null     object 
 2   metadata.participants    86 non-null     object 
 3   info.gameCreation        86 non-null     float64
 4   info.gameDuration        86 non-null     float64
 5   info.gameEndTimestamp    86 non-null     float64
 6   info.gameId              86 non-null     float64
 7   info.gameMode            86 non-null     object 
 8   info.gameName            86 non-null     object 
 9   info.gameStartTimestamp  86 non-null     float64
 10  info.gameType            86 non-null     object 
 11  info.gameVersion         86 non-null     object 
 12  info.mapId               86 non-null     float64
 13  info.participants        86 non-null     object 
 14  info.platformId          86 

In [46]:
import requests
import pandas as pd

# Riot API key
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}
# 사용자의 puuid와 match id 리스트
puuid = puuid_2
match_id_list = gamelist_2

# 각 match id에 해당하는 match 정보 가져오기
match_data_list = []
for match_id in match_id_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    match_data_list.append(response.json())

# match 정보를 데이터프레임으로 변환하기
match_df_list = []
for match_data in match_data_list:
    match_df = pd.json_normalize(match_data)
    match_df_list.append(match_df)
    
# 모든 match 정보를 하나의 데이터프레임으로 합치기
match_df_2 = pd.concat(match_df_list, axis=0)


In [47]:
import requests
import pandas as pd

# Riot API key
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}
# 사용자의 puuid와 match id 리스트
puuid = puuid_3
match_id_list = gamelist_3

# 각 match id에 해당하는 match 정보 가져오기
match_data_list = []
for match_id in match_id_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    match_data_list.append(response.json())

# match 정보를 데이터프레임으로 변환하기
match_df_list = []
for match_data in match_data_list:
    match_df = pd.json_normalize(match_data)
    match_df_list.append(match_df)
    
# 모든 match 정보를 하나의 데이터프레임으로 합치기
match_df_3 = pd.concat(match_df_list, axis=0)


In [50]:
import requests
import pandas as pd

# Riot API key
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}
# 사용자의 puuid와 match id 리스트
puuid = puuid_4
match_id_list = gamelist_4

# 각 match id에 해당하는 match 정보 가져오기
match_data_list = []
for match_id in match_id_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    match_data_list.append(response.json())

# match 정보를 데이터프레임으로 변환하기
match_df_list = []
for match_data in match_data_list:
    match_df = pd.json_normalize(match_data)
    match_df_list.append(match_df)
    
# 모든 match 정보를 하나의 데이터프레임으로 합치기
match_df_4 = pd.concat(match_df_list, axis=0)


In [51]:
import requests
import pandas as pd

# Riot API key
api_key = 'RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d'
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}
# 사용자의 puuid와 match id 리스트
puuid = puuid_5
match_id_list = gamelist_5

# 각 match id에 해당하는 match 정보 가져오기
match_data_list = []
for match_id in match_id_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    match_data_list.append(response.json())

# match 정보를 데이터프레임으로 변환하기
match_df_list = []
for match_data in match_data_list:
    match_df = pd.json_normalize(match_data)
    match_df_list.append(match_df)
    
# 모든 match 정보를 하나의 데이터프레임으로 합치기
match_df_5 = pd.concat(match_df_list, axis=0)


In [58]:
lol_match_data = pd.concat([match_df_1,match_df_2,match_df_3, match_df_4 ,match_df_5],axis=0)

In [63]:
lol_match_data

,metadata.dataVersion,metadata.matchId,metadata.participants,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.participants,info.platformId,info.queueId,info.teams,info.tournamentCode,status.message,status.status_code
0,2,KR_6407238339,[MsiZlkhRXHP2WSbqEGF85QbYhhWJY2_ps4rrIsQhqBzJa...,1.678941e+12,107.0,1.678941e+12,6.407238e+09,CLASSIC,teambuilder-match-6407238339,1.678941e+12,MATCHED_GAME,13.5.495.8836,11.0,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440.0,"[{'bans': [{'championId': -1, 'pickTurn': 1}, ...",,NaN,NaN
0,2,KR_6407219214,[MsiZlkhRXHP2WSbqEGF85QbYhhWJY2_ps4rrIsQhqBzJa...,1.678939e+12,1464.0,1.678940e+12,6.407219e+09,CLASSIC,teambuilder-match-6407219214,1.678939e+12,MATCHED_GAME,13.5.495.8836,11.0,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440.0,"[{'bans': [{'championId': 350, 'pickTurn': 1},...",,NaN,NaN
0,2,KR_6401412528,[TCQwnu3pDofBuF42GmUaC90nN-ir-71p6nxclyPiOCSuD...,1.678547e+12,938.0,1.678548e+12,6.401413e+09,CLASSIC,teambuilder-match-6401412528,1.678547e+12,MATCHED_GAME,13.5.495.8836,11.0,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440.0,"[{'bans': [{'championId': 517, 'pickTurn': 1},...",,NaN,NaN
0,2,KR_6401305429,[aGnXwyWnyD06XnbV46zIugKRWpLG_747FKchzVaB1CYQI...,1.678544e+12,2427.0,1.678547e+12,6.401305e+09,CLASSIC,teambuilder-match-6401305429,1.678544e+12,MATCHED_GAME,13.5.495.8836,11.0,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440.0,"[{'bans': [{'championId': 45, 'pickTurn': 1}, ...",,NaN,NaN
0,2,KR_6401213810,[l4Hs1khAyvm015no-avuc6PYTv8YRXSfYrPG4_pWfQdp0...,1.678542e+12,1722.0,1.678544e+12,6.401214e+09,CLASSIC,teambuilder-match-6401213810,1.678542e+12,MATCHED_GAME,13.5.495.8836,11.0,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440.0,"[{'bans': [{'championId': 517, 'pickTurn': 1},...",,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2,KR_6386144872,[GAa-D9LWAuPahjo3C62fe7bQkIYj2K4gbF9enjtgZ2Pui...,1.677640e+12,1707.0,1.677642e+12,6.386145e+09,CLASSIC,teambuilder-match-6386144872,1.677640e+12,MATCHED_GAME,13.4.494.1549,11.0,"[{'allInPings': 0, 'assistMePings': 8, 'assist...",KR,420.0,"[{'bans': [{'championId': 238, 'pickTurn': 1},...",,NaN,NaN
0,2,KR_6386114340,[lV_4CL7yEYfJEtVlfC5bXU2kd3ayrZR_C8MQCGqs0J4_2...,1.677637e+12,2136.0,1.677639e+12,6.386114e+09,CLASSIC,teambuilder-match-6386114340,1.677637e+12,MATCHED_GAME,13.4.494.1549,11.0,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,420.0,"[{'bans': [{'championId': 22, 'pickTurn': 1}, ...",,NaN,NaN
0,2,KR_6386089059,[7d0er7pHqlm7VrahV0YPfWa1EFV-jXF1azEke6xWsFQnR...,1.677635e+12,1650.0,1.677636e+12,6.386089e+09,CLASSIC,teambuilder-match-6386089059,1.677635e+12,MATCHED_GAME,13.4.494.1549,11.0,"[{'allInPings': 0, 'assistMePings': 2, 'assist...",KR,420.0,"[{'bans': [{'championId': 84, 'pickTurn': 1}, ...",,NaN,NaN
0,2,KR_6386065665,[wuxCd8IMVvLLm1wBuqCWYs_k99K9VW2hkRRvEv4bwpe04...,1.677633e+12,1332.0,1.677634e+12,6.386066e+09,CLASSIC,teambuilder-match-6386065665,1.677633e+12,MATCHED_GAME,13.4.494.1549,11.0,"[{'allInPings': 0, 'assistMePings': 1, 'assist...",KR,420.0,"[{'bans': [{'championId': 53, 'pickTurn': 1}, ...",,NaN,NaN


In [62]:
lol_match_data.to_csv('lol_match_data.csv', header=False, index=False)

In [ ]:
import pandas as pd
import requests
import json

# Riot API endpoint
base_url = "https://asia.api.riotgames.com"

# Your API key
api_key = "your_api_key"

# List of user PUUIDs
puuids = ["puuid1", "puuid2", "puuid3"]

# List of match IDs for each user
match_ids = []

# Collect match IDs for each user
for puuid in puuids:
    response = requests.get(
        f"{base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids",
        headers={"X-Riot-Token": api_key},
    )
    match_ids.append(response.json())

# Create an empty DataFrame with columns for each match
columns = ["match_id", "champion", "items", "win", "lose", "kda", "role"]
df = pd.DataFrame(columns=columns)

# Loop through each user's match IDs and collect data
for i, puuid in enumerate(puuids):
    for match_id in match_ids[i]:
        response = requests.get(
            f"{base_url}/lol/match/v5/matches/{match_id}",
            headers={"X-Riot-Token": api_key},
        )
        match_data = response.json()

        # Extract relevant data from match data
        champion = ...
        items = ...
        win = ...
        lose = ...
        kda = ...
        role = ...

        # Add data to DataFrame as a new row
        new_row = pd.DataFrame(
            [[match_id, champion, items, win, lose, kda, role]],
            columns=columns,
        )
        df = df.append(new_row, ignore_index=True)


# 0321(화) 여러명의 매치 데이터 생성하기 

In [11]:
import requests
import pandas as pd

# Set your API key and the endpoint you want to use
api_key = "RGAPI-51c34da8-fcfc-4c82-8ebd-354fb62799cf"
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-f7b1583f-6a39-4310-8fdd-6c02d853616d"

}
endpoint = "https://kr.api.riotgames.com/lol/match/v5/matches/{match_id}"

# Set the list of summoner names you want to analyze
summoner_names = ["J꾸짖을 갈", "푸 투", "마법소녀 모야삡", "응애소나", "막타의호흡"]

# Get the account IDs for the players you want to analyze
account_ids = []
for summoner_name in summoner_names:
    region = "kr"
    url = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}"*******CHECK HERE!!!
    response = requests.get(url)
    account_id = response.json()["accountId"]
    account_ids.append(account_id)
----
# 사용자의 puuid와 match id 리스트
puuid = puuid_5
match_id_list = gamelist_5

# 각 match id에 해당하는 match 정보 가져오기
match_data_list = []
for match_id in match_id_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    match_data_list.append(response.json())
----
# Set the query parameters for the match list
params = {
    "queue": 420,  # Only get data for Ranked Solo/Duo games
    "season": 13,  # Only get data for Season 13
    "champion": None,  # No champion filtering
    "beginIndex": 0,  # Start at the first match
    "endIndex": 100,  # Get the first 100 matches
}

# Create an empty dataframe to store the match data
df = pd.DataFrame(columns=["match_id", "win", "champ_name", "kills", "deaths", "assists", "kda", "role", "lane"])

# Loop over the account IDs and get the match data for each one
for account_id in account_ids:
    # Get the match list for the specified account ID and query parameters
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matchlists/by-account/{account_id}"
    response = requests.get(url, params=params, headers=request_header)
    
    # Parse the response and create a list of match IDs
    match_list = response.json()["matches"]
    match_ids = [match["gameId"] for match in match_list]

    # Loop over the match IDs and get the match data for each one
    for match_id in match_ids:
        url = endpoint.format(match_id=match_id)
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        data = response.json()
        win = None
        champ_name = None
        k = None
        d = None
        a = None
        kda = None
        role = None
        lane = None
        for participant in data["participants"]:
            if participant["participantId"] == data["participantIdentities"][0]["participantId"]:
                win = participant["stats"]["win"]
                champ_name = participant["championId"]
                k = participant["stats"]["kills"]
                d = participant["stats"]["deaths"]
                a = participant["stats"]["assists"]
                kda = (k + a) / max(d, 1)
                role = participant["timeline"]["role"]
                lane = participant["timeline"]["lane"]
                break
        if win is not None and data["queueId"] == 420 and data["seasonId"] == 13 and champ_name is not None and \
                data["participants"][0]["highestAchievedSeasonTier"] == "GOLD" and role == "SOLO" and lane == "MIDDLE":
            df = df.append({"match_id": match_id, "win": win, "champ_name": champ_name})

KeyError: ignored

In [12]:
print(response.content)

b'{"status":{"message":"Forbidden","status_code":403}}'
